# Test anomaly-score-unsupervised
This notebook smoketests the anomaly-score-unsupervised component

In [ ]:
# Start the anomaly-score-unsupervised notebook using ipython. This will create a local service on port 8080

import subprocess
sp  = subprocess.Popen(["ipython","../anomaly-score-unsupervised.ipynb"])
sp

In [ ]:
# Give the service some time to start

import time

print("Waiting for service to start...")
time.sleep(10)
print("..hope it is started now...")


In [ ]:
import requests
import json
import pickle


In [ ]:
# e-serialize test data

with open('watsoniotp.healthy.phase_aligned.pickle','rb') as file_object:
    raw_data = file_object.read()
    data_healthy = pickle.loads(raw_data, encoding='latin1')

with open('watsoniotp.broken.phase_aligned.pickle','rb') as file_object:
    raw_data = file_object.read()
    data_broken = pickle.loads(raw_data, encoding='latin1')

In [ ]:
# inspect 3D time series of healthy data

import matplotlib.pyplot as plt

fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
size = len(data_healthy)
#ax.set_ylim(0,energy.max())
ax.plot(range(0,size), data_healthy[:,0], '-', color='blue', animated = True, linewidth=1)
ax.plot(range(0,size), data_healthy[:,1], '-', color='red', animated = True, linewidth=1)
ax.plot(range(0,size), data_healthy[:,2], '-', color='green', animated = True, linewidth=1)

In [ ]:
# inspect 3D time series of broken data

import matplotlib.pyplot as plt

fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
size = len(data_broken)
#ax.set_ylim(0,energy.max())
ax.plot(range(0,size), data_broken[:,0], '-', color='blue', animated = True, linewidth=1)
ax.plot(range(0,size), data_broken[:,1], '-', color='red', animated = True, linewidth=1)
ax.plot(range(0,size), data_broken[:,2], '-', color='green', animated = True, linewidth=1)

In [ ]:
# prepare for frequency spectrum comparison by applying fft

import numpy as np
data_healthy_fft = np.fft.fft(data_healthy)
data_broken_fft = np.fft.fft(data_broken)

In [ ]:
# plot frequency spectrum for healthy data

fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
size = len(data_healthy_fft)
ax.plot(range(0,size), data_healthy_fft[:,0].real, '-', color='blue', animated = True, linewidth=1)
ax.plot(range(0,size), data_healthy_fft[:,1].imag, '-', color='red', animated = True, linewidth=1)
ax.plot(range(0,size), data_healthy_fft[:,2].real, '-', color='green', animated = True, linewidth=1)

In [ ]:
# plot frequency spectrum for broken data

fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
size = len(data_healthy_fft)
ax.plot(range(0,size), data_broken_fft[:,0].real, '-', color='blue', animated = True, linewidth=1)
ax.plot(range(0,size), data_broken_fft[:,1].imag, '-', color='red', animated = True, linewidth=1)
ax.plot(range(0,size), data_broken_fft[:,2].real, '-', color='green', animated = True, linewidth=1)

In [ ]:
# convert test data from numpy to json (to be used when calling th web service via HTTP)

data_healthy = data_healthy.tolist()
data_healthy = json.dumps(data_healthy)
data_broken = data_broken.tolist()
data_broken = json.dumps(data_broken)

In [ ]:
# initialize http client parameters
url = "http://localhost:8080/send_data"
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

In [ ]:
# test with healthy data
for _ in range(3):
    r = requests.post(url, data=data_healthy, headers=headers)
    assert(r.status_code==200)

In [ ]:
#test with broken data

r = requests.post(url, data=data_broken, headers=headers)
assert(r.status_code==200)

In [ ]:
# retest with healthy data

for _ in range(3):
    r = requests.post(url, data=data_healthy, headers=headers)
    assert(r.status_code==200)

In [ ]:
# display loss (reconstruction) history

from IPython.display import Image
Image("http://localhost:8080/get_loss_as_image.png")

In [ ]:
# stop service process

import psutil

pids = psutil.pids()

for pid in pids:
    if psutil.pid_exists(pid):
        p = psutil.Process(pid)
        if any("anomaly-score-unsupervised.ipynb" in s for s in p.cmdline()):
            print(p.kill())